In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import bezier
import functools
%load_ext autoreload
%autoreload 2

In [ ]:
class BezierCurve(object):
    
    def __init__(
        self,
        nodes=None,
        degree=None,
        n_eval_points=100,
    ):
        nodes = nodes.transpose(np.argsort(np.array(nodes.shape)-2))  # hacky, to get in right orientation
        self._nodes = nodes
        self._degree = degree
        self.n_eval_points = n_eval_points
        
    @property
    def degree(self):
        if self._degree is None:
            self._degree = self.nodes.shape[1]-1
        return self._degree
    
    @property
    def nodes(self):
        return self._nodes
    
    @property
    def _fortran_nodes(self):
        return np.asfortranarray(self.nodes)
    
    @property
    def _curve(self):
        return bezier.Curve(self._fortran_nodes, self.degree)
    
    @property
    def eval_points(self):
        return np.linspace(0, 1, self.n_eval_points)
    
    @property
    def evaluated_curve(self):
        x, y = self._curve.evaluate_multi(self.eval_points)
        return np.stack([x, y]).T
    
    @property
    def linestring(self):
        return LineString(self.evaluated_curve)

In [ ]:
class PerlinGrid(object):

    def __init__(self, poly, lod=4, falloff=None, noiseSeed=71, noise_scale=0.001, output_range=(0, np.pi*2)):

        self.p = poly

        self.vsk = vsketch.Vsketch()
        self.lod = lod
        self.falloff = falloff
        self.noiseSeed = noiseSeed
        self.noise_scale = noise_scale
        self.vsk.noiseSeed(self.noiseSeed)
        self.vsk.noiseDetail(lod=self.lod, falloff=self.falloff)
        self.output_range = output_range
        
    def noise(self, x, y):
        x = x * self.noise_scale
        y = y * self.noise_scale
        output = self.vsk.noise(x=x, y=y)
        return np.interp(output, [0, 1], self.output_range)


# Cell
class Particle(object):

    def __init__(self, pos, grid, stepsize=1):
        self.pos = Point(pos)
        self.grid = grid
        self.stepsize = stepsize
        self.n_step = 0
        self.pts = [self.pos]
        self.in_bounds = True

    @property
    def x(self):
        return self.pos.x
    
    @property
    def y(self):
        return self.pos.y
    
    @property
    def xy(self):
        return np.array([self.x, self.y])

    @property
    def line(self):
        return LineString(self.pts)

    def get_angle(self):
        self.a = self.grid.noise(x=self.x, y=self.y)

    def check_if_in_bounds(self):
        self.in_bounds = self.grid.p.contains(self.pos)

    def calc_step(self):
        self.get_angle()
        self.dx = np.cos(self.a) * self.stepsize
        self.dy = np.sin(self.a) * self.stepsize


    def step(self):
        self.check_if_in_bounds()
        if self.in_bounds:
            self.calc_step()
            self.pos = sa.translate(self.pos, xoff=self.dx, yoff=self.dy)
            self.pts.append(self.pos)

In [ ]:
paper_size = '11x14 inches'
border:float=20
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:

poly = drawbox
pg = PerlinGrid(poly, lod=10, falloff=None, noise_scale=0.043, noiseSeed=5)

In [ ]:
start_point = pg.p.centroid
start_buffer = 60
n_nodes = 400
n_steps = 60
circ = start_point.buffer(start_buffer).boundary
pts = [circ.interpolate(d, normalized=True) for d in np.linspace(0., 0.99, n_nodes)]
particles = [Particle(pos=pos, grid=pg, stepsize=1) for pos in pts if pg.p.contains(pos)]
repel_scale = 30


linestrings = []
_ls = LineString([p.pos for p in particles])
nodes = np.array(_ls)
ls = BezierCurve(nodes).linestring
linestrings.append(ls)

In [ ]:
all_pts = []
for ii in tqdm(range(n_steps)):
    for p in particles:
        
        p.step()
        pvec = (np.array(p.pos) - np.array(start_point))
        norm_pvec = pvec /np.linalg.norm(pvec)
        norm_pvec * repel_scale
        xoff, yoff = norm_pvec
#         xoff += 1
#         yoff -= 1
        p.pos = sa.translate(p.pos, xoff=xoff, yoff=yoff)
    all_pts += [p.pos for p in particles]
    
    
    

In [ ]:
layer = MultiLineString(linestrings)

layer = gp.make_like(layer, drawbox)

In [ ]:
lb = MultiPoint(all_pts).buffer(0.05, join_style=2, cap_style=1).boundary

In [ ]:
lb = layer.buffer(0.05, join_style=2, cap_style=2).boundary

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.stroke(1)
sk.geometry(lb)
sk.display(color_mode='layer')

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0342_perlin_repeater_buffered_less.svg'

sk.save(savepath)

## try 2

In [ ]:
paper_size = '11x14 inches'
border:float=35
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:

poly = drawbox
pg = PerlinGrid(poly, xstep=1, ystep=1, lod=10, falloff=None, noise_scale=0.053, noiseSeed=5)

In [ ]:
start_point = pg.p.centroid
start_buffer = 10
n_nodes = 400
n_steps = 70
circ = start_point.buffer(start_buffer).boundary
pts = [circ.interpolate(d, normalized=True) for d in np.linspace(0., 1., n_nodes)]
particles = [Particle(pos=pos, grid=pg, stepsize=1) for pos in pts if pg.p.contains(pos)]
    


linestrings = []
_ls = LineString([p.pos for p in particles])
nodes = np.array(_ls)
ls = BezierCurve(nodes).linestring
linestrings.append(ls)

for ii in tqdm(range(n_steps)):
    for p in particles:
        p.pos = sa.translate(p.pos, xoff=1.3, yoff=0.4)
        p.step()

    _ls = LineString([p.pos for p in particles])
    nodes = np.array(_ls)
    ls = BezierCurve(nodes = nodes).linestring
    
    linestrings.append(ls)

layer = MultiLineString(linestrings)

layer = gp.make_like(layer, drawbox)

In [ ]:
buffer_gen = ss.uniform(loc=1, scale=6).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-0.8, high=-0.2)
angles_gen = ss.uniform(loc=0, scale=360).rvs
angles_gen = gp.make_callable(80)
d_translate_factor_gen = ss.uniform(loc=0.2, scale=0.6).rvs

In [ ]:
fills = []
all_polys = Polygon()
for l in layer:
    p = l.buffer(0.5, cap_style=2, join_style=3)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.15, 0.15, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

In [ ]:
L1 = gp.merge_LineStrings([f for f in fills if f.length > 0.1])

In [ ]:
buffer_gen = ss.uniform(loc=1, scale=6).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-2, high=-1.2)
angles_gen = ss.uniform(loc=0, scale=360).rvs
angles_gen = gp.make_callable(80)
d_translate_factor_gen = ss.uniform(loc=0.2, scale=0.6).rvs

In [ ]:
fills = []
all_polys = Polygon()
for l in layer:
    p = l.buffer(0.5, cap_style=2, join_style=3)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.15, 0.15, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

In [ ]:
L2 = gp.merge_LineStrings([f for f in fills if f.length > 0.1])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.stroke(1)
sk.geometry(L1)
sk.stroke(2)
sk.geometry(L1)
sk.vpype('linesimplify linemerge linesort')
sk.display(color_mode='layer')

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0345_perlin_repeater_buffered_fills_2color.svg'

sk.save(savepath)

## try 2

In [ ]:
paper_size = '11x14 inches'
border:float=35
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:

poly = drawbox
pg = PerlinGrid(poly, xstep=1, ystep=1, lod=10, falloff=None, noise_scale=0.053, noiseSeed=5)

In [ ]:
start_point = pg.p.centroid
start_buffer = 10
n_nodes = 400
n_steps = 70
circ = start_point.buffer(start_buffer).boundary
pts = [circ.interpolate(d, normalized=True) for d in np.linspace(0., 1., n_nodes)]
particles = [Particle(pos=pos, grid=pg, stepsize=1) for pos in pts if pg.p.contains(pos)]
    


linestrings = []
_ls = LineString([p.pos for p in particles])
nodes = np.array(_ls)
ls = BezierCurve(nodes).linestring
linestrings.append(ls)

for ii in tqdm(range(n_steps)):
    for p in particles:
        p.pos = sa.translate(p.pos, xoff=1.3, yoff=0.4)
        p.step()

    _ls = LineString([p.pos for p in particles])
    nodes = np.array(_ls)
    ls = BezierCurve(nodes = nodes).linestring
    
    linestrings.append(ls)

layer = MultiLineString(linestrings)

layer = gp.make_like(layer, drawbox)

In [ ]:
layer = layer.buffer(0.01, join_style=2, cap_style=2).buffer(-0.01).boundary

In [ ]:
layer = [l for l in layer if l.length>0.1]

In [ ]:
buffer_gen = ss.uniform(loc=1, scale=6).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-0.8, high=-0.2)
angles_gen = ss.uniform(loc=0, scale=360).rvs
angles_gen = gp.make_callable(80)
d_translate_factor_gen = ss.uniform(loc=0.2, scale=0.6).rvs

In [ ]:
fills = []
all_polys = Polygon()
for l in tqdm(layer):
    p = l.buffer(0.5, cap_style=2, join_style=3)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.15, 0.15, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

In [ ]:
L1 = gp.merge_LineStrings([f for f in fills if f.length > 0.1])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.stroke(1)
sk.geometry(L1)
# sk.stroke(2)
# sk.geometry(L1)
sk.vpype('linesimplify linemerge linesort')
sk.display(color_mode='layer')

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0346_perlin_repeater_buffered_fills_spikedout.svg'

sk.save(savepath)

## try 2

In [ ]:
paper_size = '11x14 inches'
border:float=35
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:

poly = drawbox
pg = PerlinGrid(poly, xstep=1, ystep=1, lod=10, falloff=None, noise_scale=0.053, noiseSeed=5)

In [ ]:
start_point = pg.p.centroid
start_buffer = 6
n_nodes = 200

circ = start_point.buffer(start_buffer).boundary
pts = [circ.interpolate(d, normalized=True) for d in np.linspace(0.1, 0.99, n_nodes)]
particles = [Particle(pos=pos, grid=pg, stepsize=1) for pos in pts if pg.p.contains(pos)]
    


_ls = LineString([p.pos for p in particles])
nodes = np.array(_ls)
ls = BezierCurve(nodes).linestring
linestrings.append(ls)

In [ ]:
n_steps = 80
linestrings = []
for ii in tqdm(range(n_steps)):
    for p in particles:
        p.pos = sa.translate(p.pos, xoff=0., yoff=1)
        p.step()

    _ls = LineString([p.pos for p in particles])
    nodes = np.array(_ls)
    ls = BezierCurve(nodes = nodes).linestring
    
    linestrings.append(ls)

layer = MultiLineString(linestrings)

layer = gp.make_like(layer, drawbox)

In [ ]:
d=1e-1
mlayer = layer.buffer(d, join_style=1, cap_style=1).buffer(-d).boundary

In [ ]:
mlayer = [l for l in mlayer if l.length>0.01]

In [ ]:
buffer_gen = ss.uniform(loc=1, scale=6).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-0.8, high=-0.2)
angles_gen = ss.uniform(loc=0, scale=360).rvs
angles_gen = gp.make_callable(80)
d_translate_factor_gen = ss.uniform(loc=0.2, scale=0.6).rvs

In [ ]:
buffer_gen = functools.partial(np.interp, xp=[-2, 1], fp=[1,3])
d_buffer_gen = functools.partial(np.interp, xp=[-1, 1], fp=[-0.2, -0.7])

In [ ]:
fills = []
all_polys = Polygon()
for l in tqdm(mlayer):
#     p = l.buffer(0.5, cap_style=2, join_style=3)
    p = l.buffer(buffer_gen(np.log10(l.length)), cap_style=2, join_style=2)
    
    stp = gp.ScaleTransPrms(
        d_buffer=d_buffer_gen(np.log10(l.length)),
        angles=angles_gen(),
        d_translate_factor=d_translate_factor_gen(), 
        n_iters=300)
    stp.d_buffers += np.random.uniform(-0.15, 0.15, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

In [ ]:
L2 = gp.merge_LineStrings([f for f in fills if f.length > 0.1])

In [ ]:
n_steps = 55
linestrings = []
for ii in tqdm(range(n_steps)):
    for p in particles:
        p.pos = sa.translate(p.pos, xoff=0., yoff=0.6)
        p.step()

    _ls = LineString([p.pos for p in particles])
    nodes = np.array(_ls)
    ls = BezierCurve(nodes = nodes).linestring
    
    linestrings.append(ls)

layer = MultiLineString(linestrings)

layer = gp.make_like(layer, drawbox)

In [ ]:
buffer_gen = ss.uniform(loc=1, scale=6).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-0.8, high=-0.2)
angles_gen = ss.uniform(loc=0, scale=360).rvs
angles_gen = gp.make_callable(80)
d_translate_factor_gen = ss.uniform(loc=0.2, scale=0.6).rvs

In [ ]:
fills = []
# all_polys = Polygon()
for l in tqdm(layer):
    p = l.buffer(0.5, cap_style=2, join_style=3)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.15, 0.15, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

In [ ]:
L1 = gp.merge_LineStrings([f for f in fills if f.length > 0.1])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.stroke(1)
sk.geometry(L1)
sk.stroke(2)
sk.geometry(L2)
sk.vpype('linesimplify linemerge linesort')
sk.display(color_mode='layer')

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0348_perlin_repeater_buffered_fills_spikedout_2col.svg'

sk.save(savepath)